https://dev.twitter.com/rest/reference/get/followers/ids

Returns variety of information of the followers of a user specified by the required user_id or screen_name parameter.

1. You need to enter your API credentials to the key list.
2. Enter the user_id or screen_name ofthe user whoose followers will be fetched.
3. You can specify which info you need to the info list. You can find the name of the returned parameters (id, followers_count etc.) in the link above.
4. Script outputs the resulting data to followers_info_out.txt as a csv formatted file

In [15]:
from birdy.twitter import UserClient, BirdyException 
from time import sleep
import string

key = [
    ["Consumer Key", "Consumer Secret", "Access Token", "Access Token Secret"]
]

client = UserClient(key[0][0], key[0][1], key[0][2], key[0][3])

In [17]:
uid = 0
sname = ''

info = [
    'id',
    'screen_name',
    'followers_count',
    'statuses_count'
]

In [18]:
followerIDs = []

keyInd = 0
client = UserClient(key[keyInd][0], key[keyInd][1], key[keyInd][2], key[keyInd][3])

protec = False

while(True):
    try:
        response = client.api.followers.ids.get(user_id = uid, screen_name = sname, count=5000, cursor=-1, stringify_ids=True)
        break
    except Exception as err:
        print(err.status_code)
        print(err)
        if err.status_code == 429:
            sleep(60)
            keyInd = (keyInd + 1)%len(key)
        else:
            protec = True
            break
        client = UserClient(key[keyInd][0], key[keyInd][1], key[keyInd][2], key[keyInd][3])
        
if protec:
    followerIDs.append([])
    print('protected!')
else:
    ncur = response.data['next_cursor']
    for s in response.data['ids']:
        followerIDs.append(s)

    while(ncur != 0):
        while(True):
            try:
                response = client.api.followers.ids.get(user_id = uid, screen_name = sname, count=5000, cursor=ncur, stringify_ids=True)
                break
            except Exception as err:
                print(err.status_code)
                print(err)
                if err.status_code == 429:
                    sleep(60)
                    keyInd = (keyInd + 1)%len(key)
                else:
                    sleep(15)
                client = UserClient(key[keyInd][0], key[keyInd][1], key[keyInd][2], key[keyInd][3])

        ncur = response.data['next_cursor']
        for s in response.data['ids']:
            followerIDs.append(s)

In [19]:
userInfo = []

for i in range(int(len(followerIDs)/100)+1):
    imin = i*100
    imax = (i+1)*100
    if i == int(len(followerIDs)/100):
        imax = len(followerIDs)
    while(True):
        try:
            response = client.api.users.lookup.get(user_id=followerIDs[imin : imax])
            for rd in response.data:
                tmpInfo = []
                for i in info:
                    tmpInfo.append(rd[i])
                userInfo.append(tmpInfo)
            break
        except Exception as err:
            print(err.status_code)
            print(err)
            if err.status_code == 429:
                sleep(60)
                keyInd = (keyInd + 1)%len(key)
            else:
                break
            client = UserClient(key[keyInd][0], key[keyInd][1], key[keyInd][2], key[keyInd][3])

In [20]:
filename = 'followers_info_out.txt'
f = open(filename, 'w', encoding='utf-8')

f.write(str(info[0]))
for i in info[1:]:
    f.write(',')
    f.write(str(i))
f.write("\n")

for ui in userInfo:
    f.write(str(ui[0]))
    for i in ui[1:]:
        f.write(',')
        f.write(str(i))
    f.write("\n")
f.close()